In [1]:
from textlong import idea
from langchain_zhipu import ChatZhipuAI, ZhipuAIEmbeddings
from textlong.knowledge import LocalFilesLoader

In [2]:
qa = LocalFilesLoader(["QA", "工美港", "汇报资料"])
# qa.load()
qa.get_files()

['./QA/demo.xlsx', './工美港/消防资料.md', '现状总结.md']

In [3]:
model = ZhipuAIEmbeddings()
qa.cache_embeddings(model)

<_工美港_消防资料_md> 低层建筑 
场景：大于300平方米的银行营业厅;问题：是否需要设置自动喷水系统？
<_工美港_消防资料_md> 低层建筑 
场景：大于300平方米的银行营业厅&#10;问题：&#10;（1）是不是人员密集场所？&...
<_工美港_消防资料_md> 低层建筑 
场景：5层高的大学实验楼&#10;（主要是化工、物理等实验室，并且采用了分体式空调）&#...
<_工美港_消防资料_md> 工美港基本资料：有320个消防阀门，正常水压的阀门有300个，其余略高但属于可控范围，分布在3个隔离...
Cached 4 embeddings to ./__CACHE_EMBEDDINGS__/ !


True

In [5]:
from langchain_community.vectorstores import FAISS
from textlong.memory import MemoryManager
from textlong.knowledge import collect_docs
from textlong.chain import create_qa_chain

db = FAISS.from_embeddings(*qa.load_embeddings(model))
retriever = db.as_retriever()
memory = MemoryManager()
chain = create_qa_chain(ChatZhipuAI(), db.as_retriever(), memory)

In [7]:
for x in chain.stream({"input": "工美港有多少正常的消防阀门？"}):
    print(x.content, end="")

问题答案：300个。相关规范解释：工美港基本资料表明，共有320个消防阀门，其中正常水压的阀门有300个，符合正常消防安全标准。

In [6]:
for x in chain.stream({"input": "辣子鸡怎么做？"}):
    print(x.content, end="")

我只回答与消防领域相关的问题，请重新提问。

In [8]:
chain.get_prompts()[0]

PromptTemplate(input_variables=['context', 'question'], partial_variables={'history': ''}, template='你是一名咨询专家，只负责根据资料回答相关提问，禁止回答与此无关的问题。\n\n你在回答时必须遵循以下约束：\n1. 你只能基于参考资料回答问题，务必注意资料的真实性，不要做任何编造\n2. 请使用简洁的语言回答，不要啰嗦\n3. 不要生成"根据提供的资料..."等字眼\n\n>>>>>>>>>>>>>> 你的参考资料：\n{{{context}}}\n\n>>>>>>>>>>>>>> 你必须按照如下格式输出：\n问题答案：xxx。\n\n相关规范解释：xxxxxxxx。\n\n>>>>>>>>>>>>>> 之前的对话如下：\n{{{history}}}\n\n我的问题是：{{question}}\n\n你的回答：', template_format='mustache')

In [13]:
memory.get_memory("default")

[HumanMessage(content='银行大厅要不要设置喷淋？'),
 AIMessageChunk(content='问题答案：需要设立自动喷水灭火系统。\n\n相关规范解释：《建筑设计防火规范》（GB 50016-2014）第5.3.1条规定，公共建筑中的人员密集场所应设置自动喷水灭火系统。银行营业厅作为公共场所，且面积大于300平米，符合此规范要求。', response_metadata={'finish_reason': 'stop', 'model': 'glm-4', 'created': 1719388633, 'index': 0, 'usage': {'prompt_tokens': 960, 'completion_tokens': 68, 'total_tokens': 1028}}, id='8783895246509809662'),
 HumanMessage(content='沙区纪委平台数据来自哪里？'),
 AIMessageChunk(content='问题答案：沙区纪委平台数据来自“新渝快办”系统开发商阿里提供的数据，以及与市纪委平台的数据共享。\n\n相关规范解释：《市纪委公权力监督平台现状分析及发展计划》提出，沙区纪委平台在技术升级与数据共享方面，通过优化数据抽取逻辑和建立数据共享机制，确保监督数据的有效获取和实时同步。沙区纪委同时保留现有系统，并从市纪委平台获取与监督相关的数据。', response_metadata={'finish_reason': 'stop', 'model': 'glm-4', 'created': 1719388647, 'index': 0, 'usage': {'prompt_tokens': 2054, 'completion_tokens': 100, 'total_tokens': 2154}}, id='8783894146998085616')]

In [8]:
from textlong.hub import load_prompt, clone_prompt

In [9]:
clone_prompt("RAG", tag="chat")

True

In [10]:
%pwd

'/Users/xuehongwei/github/textlong/notes/00 howto'

In [18]:
print(load_prompt("RAG", tag="chat").format(
    question="有几个隔离区？",
    context="工美港基本资料：有320个消防阀门，正常水压的阀门有300个，其余略高但属于可控范围，分布在3个隔离区。"))

你是一名咨询专家，只负责根据资料回答相关提问，禁止回答与消防领域无关的问题。

你在回答时必须遵循以下约束：
1. 你只能基于参考资料回答问题，务必注意资料的真实性，不要做任何编造
2. 请使用简洁的语言回答，不要啰嗦
3. 不要生成"根据提供的资料..."等字眼

>>>>>>>>>>>>>> 你的参考资料：
工美港基本资料：有320个消防阀门，正常水压的阀门有300个，其余略高但属于可控范围，分布在3个隔离区。

>>>>>>>>>>>>>> 你必须按照如下格式输出：
例子1（如果提问与消防领域）:

我只回答与消防领域相关的问题，请重新提问。

例子2（其他问题）：

问题答案：xxx。
相关规范解释：xxxxxxxx。

>>>>>>>>>>>>>> 之前的对话如下：


我的问题是：有几个隔离区？

你的回答：
